In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data=pd.read_csv('training_twitter.csv',delimiter=',',skipinitialspace=True)

In [3]:
train_data.drop(["tweet_id","airline","airline_sentiment_gold","name","negativereason_gold","retweet_count","tweet_coord","tweet_created","tweet_location","user_timezone"],inplace=True,axis=1)

In [4]:
y_train=[]
for i in train_data["airline_sentiment"]:
    y_train.append(i)
x_train=[]
for j in train_data["text"]:
    x_train.append(j)

In [5]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
stop= stopwords.words('english')
punctuations=list(string.punctuation)
stop=stop+punctuations
lemmatizer= WordNetLemmatizer()

In [7]:
def data_cleaning(data):
   clean_data=[]
   for i in range (len(data)):
      words=word_tokenize(data[i].lower())
      #remove_stopwords=[w for w in words if not w in stop]
      clean=[]
      for w in words:
         if w.lower() not in stop:
            pos=pos_tag([w])
            clean_words=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            clean.append(clean_words.lower())
      clean_data.append(clean)
   return clean_data

In [8]:
train_clean_data = data_cleaning(x_train)
train_documents=[" ".join(document) for document in train_clean_data]

In [9]:
test_data=pd.read_csv('test_twitter.csv',delimiter=',',skipinitialspace=True)
test_data.drop(["tweet_id","airline","airline_sentiment_gold","name","negativereason_gold","retweet_count","tweet_coord","tweet_created","tweet_location","user_timezone"],inplace=True,axis=1)
x_test=[]
for j in test_data["text"]:
    x_test.append(j)

test_clean_data = data_cleaning (x_test)
test_documents=[" ".join(document)for document in test_clean_data]

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer (max_features=50000)
x_train_features=count_vector.fit_transform(train_documents)
x_test_features=count_vector.transform(test_documents)

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report 
alg=MultinomialNB()
alg.fit (x_train_features,y_train)
y_predict = alg.predict (x_test_features)
np.savetxt("twitter.csv",y_predict, fmt = '%s')
#print(confusion_matrix(y_test,y_predict))
#print(classification_matrix(y_test,y_predict))

In [40]:
alg.score(x_train_features,y_train)

0.8424408014571949